# Tic-Tac-Toe with Dependence Inversion, Open-Close and Liskov Substitution Principles

---

## Board
#### The abstract Board class allows for potential future extensions to different types of boards, which adheres to the OCP
#### Board's responsibilities are clear (managing the state and rules of the board itself), and delegate game-specific logic to another class. 
#### This would be in line with the Single Responsibility Principle (SRP).

In [ ]:
from abc import ABC, abstractmethod

class Board(ABC):
    def __init__(self, boardDimensions):
        self.boardDimensions = boardDimensions

    @abstractmethod
    def createBoard(self):
        pass

    @abstractmethod
    def printBoard(self):
        pass

    @abstractmethod
    def setCellState(self, position, state):
        pass

    @abstractmethod
    def getCellState(self, position):
        pass

    @abstractmethod
    def getBoardState(self):
        pass

    def getBoardDimensions(self):
        return self.boardDimensions

In [ ]:
class TicTacToeBoard(Board):
    def __init__(self, boardDimensions):
        super().__init__(boardDimensions)
        self.createBoard()

    def createBoard(self):
        self.boardState = {i+1: ' ' for i in range(self.boardDimensions**2)}

    def printBoard(self):
        for i in range(self.boardDimensions):
            row = [self.boardState[i*self.boardDimensions+j+1] for j in range(self.boardDimensions)]
            print('|'.join(row))
            if i < self.boardDimensions-1:
                print('-'*(self.boardDimensions*2-1))
        print('\n')

    def setCellState(self, position, state):
        self.boardState[position] = state

    def getCellState(self, position):
        return self.boardState[position]

    def getBoardState(self):
        return self.boardState

---

## Game Logic
#### Abstract class handles the game-specific logic. 
#### Placed at the same level as Board, Player, and Algorithm (directly under Game).
#### This is in line with the Single Responsibility Principle (SRP).

In [ ]:
from abc import ABC, abstractmethod

class GameLogic():
    def __init__(self, boardGame):
        self.boardGame = boardGame

    @abstractmethod
    def chkForkWin(self):
        pass

    @abstractmethod
    def chkForDraw(self):
        pass

    @abstractmethod
    def insertLetter(self, letter, position):
        pass

    @abstractmethod
    def spaceIsFree(self, position):
        pass

In [ ]:
class TicTacToeGameLogic(GameLogic):
    def __init__(self, boardGame):
        self.boardGame = boardGame

    # function to check for draw
    # check if all the spaces are taken
    def chkForDraw(self):
        boardState = self.boardGame.getBoardState()
        for key in boardState.keys():
            if boardState[key] == ' ':
                return False
        return True

    # function to check for win
    # check if any of the rows have all the same values (and not empty)
    # check if any of the columns have all the same values (and not empty)
    # check if any of the diagonals have all the same values (and not empty)
    def chkForWin(self):
        boardState = self.boardGame.getBoardState()
        boardDimensions = self.boardGame.getBoardDimensions()
        # check rows
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] != ' ':
                return True
        # check columns
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] != ' ':
                return True
        # check diagonals
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] != ' ':
            return True
        if len(set(diagonal2)) == 1 and diagonal2[0] != ' ':
            return True
        return False

    # insert letter function
    # check to see if space is free
    # if space is free, insert letter
    # if space is not free, ask user to pick a different position
    def insertLetter(self, letter, position):
        if (self.spaceIsFree(position)):
            self.boardGame.setCellState(position, letter)
            self.boardGame.printBoard()
            if (self.chkForDraw()):
                print("Draw!")
            elif (self.chkForWin()):
                print(letter + " wins!")
        else:
            print("Can't insert there!")
            # ask user to pick a different position
            position = int(input("Please enter a different position: "))
            self.insertLetter(letter, position)

    # check if space is free
    def spaceIsFree(self, position):
        cellState = self.boardGame.getCellState(position)
        if cellState == ' ':
            return True
        else:
            return False

---

## Player
#### Abstract Player class, with separate ComputerPlayer and HumanPlayer subclasses
#### Example of the OCP. Each type of Player can have its unique functionalities defined in its specific subclass. 
#### LSP is achieved by the use of a "move strategy" that is defined in the abstract Player class, and implemented in the subclasses.

In [ ]:
from abc import ABC, abstractmethod

# an abstract class for a player
class Player(ABC):
    def __init__(self, letter):
        self.letter = letter
    
    @abstractmethod
    def move(self):
        pass

In [ ]:
class HumanPlayer(Player):
    def move(self, board):
        position = int(input("Enter your move: "))
        return position

class ComputerPlayer(Player):
    def __init__(self, letter, algorithm):
        super().__init__(letter)
        self.algorithm = algorithm

    def move(self, board):
        position = self.algorithm.move(board)
        return position

---

## Algorithms
#### The abstract Algorithm class, used by the ComputerPlayer and HumanPlayer.
#### Allows for different AI strategies to be easily interchanged and potentially added in the future, following the OCP.

In [ ]:
# an abstract class for an algorithm
class Algorithm(ABC):
    def __init__(self, letter, boardGame):
        self.letter = letter
        self.boardGame = boardGame
    def getletter(self):
        return self.letter
    def setletter(self, letter):
        self.letter = letter

    @abstractmethod
    def move(self):
        pass

    # abstract method that implements the algorithm
    @abstractmethod
    def algorithm(self):
        pass

In [ ]:
class AlgorithmX(Algorithm):
    def move(self, boardGame):
        # Implement the algorithmX
        pass

class MinimaxAlgorithm(Algorithm):
    def __init__(self, letter, boardGame):
        super().__init__(letter)
        self.bestScore = 0
        self.bestMove = 0

    # function to find the best move
    def move(self):
        bestScore = -math.inf
        for key in self.boardGame.getBoardState().keys():
            if (self.boardGame.getCellState(key) == ' '):
                self.boardGame.setCellState(key, self.letter)
                score = self.algorithm(self.boardGame, 0, False)
                self.boardGame.setCellState(key, ' ')
                if (score > bestScore):
                    bestScore = score
                    self.bestMove = key
        return self.bestMove

    def algorithm(self, boardGame, depth, isMaximizing):
        # base case
        if (self.boardGame.chkForWin()):
            if (self.letter == 'X'):
                return 1
            else:
                return -1
        elif (self.boardGame.chkForDraw()):
            return 0

        if (isMaximizing):
            bestScore = -math.inf
            for key in self.boardGame.getBoardState().keys():
                if (self.boardGame.getCellState(key) == ' '):
                    self.boardGame.setCellState(key, self.letter)
                    score = self.algorithm(self.boardGame, 0, False)
                    self.boardGame.setCellState(key, ' ')
                    bestScore = max(score, bestScore)
            return bestScore
        else:
            bestScore = math.inf
            for key in self.boardGame.getBoardState().keys():
                if (self.boardGame.getCellState(key) == ' '):
                    self.boardGame.setCellState(key, self.letter)
                    score = self.algorithm(self.boardGame, 0, True)
                    self.boardGame.setCellState(key, ' ')
                    bestScore = min(score, bestScore)
            return bestScore

## Game
#### Entry Point

In [ ]:
def gameLoop():
    # Create a board
    ticTacToeBoard = TicTacToeBoard(3)
    ticTacToeBoard.printBoard()
    gameLogic = GameLogic(ticTacToeBoard)

    # Create two players
    player1 = HumanPlayer('X')
    player2 = ComputerPlayer('O', MinimaxAlgorithm('O'))

    # While the game is not over
    while not gameLogic.chkForWin() and not gameLogic.chkForDraw():
        # Player 1 move
        position = player1.move(ticTacToeBoard)
        gameLogic.insertLetter(player1.letter, position)
        if gameLogic.chkForWin() or gameLogic.chkForDraw():
            break
        # Player 2 move
        position = player2.move(ticTacToeBoard)
        gameLogic.insertLetter(player2.letter, position)

---

In [ ]:
gameLoop()